In [13]:
import os, sys, csv, json, re
import numpy as np
import pandas as pd

In [42]:
import datetime as dt
import psychropy
import ashrae_clear_sky as acs

station_name = 'KLAX'
YEAR = '2021'
START_MON = '1'
START_DAY = '1'
END_MON = '12'
END_DAY = '31'

# Input and output file location
input_folder = 'station-result'
output_folder = 'epw-results'
location = 'Los_Angeles'
LOC_WEATHER_FILE = 'epw-raw/USA_CA_Los.Angeles.Intl.AP.722950_TMY3.epw'

CSV_FILE = os.path.join(input_folder, station_name + '-' + str(YEAR) + '.csv')
EPW_FILE = os.path.join(output_folder, station_name + '-' + location + '-' + str(YEAR) + '.epw')

# to be modified according to the prepared CSV file format
csv_headers_epw_map = {6: 'dry_bulb', 7:'dew_point', 8:'rel_humi', 9:'pressure', 
                       13: 'global_hori', 14: 'dir_norm', 15: 'dif_hori', 20:'wind_dir', 21:'wind_speed'}

days_in_mon = {1:31, 2:28, 3:31, 4:30, 5:31, 6:30, 7:31, 8:31, 9:30, 10:31, 11:30, 12:31}
cloud_cover_header = 'cloud_layer_1_digit_m'

In [43]:
df_csv_result = pd.read_csv(CSV_FILE, sep=',', index_col=0, parse_dates=True, infer_datetime_format=True, encoding='UTF-8')

In [44]:
df_csv_result.head()

,date_time,dry_bulb,rel_humi,dew_point,pressure,wind_speed,wind_dir,wind_chill,heat_index,cloud_layer_1,cloud_layer_1_digit_m
0,2021-01-01-00:00,11.161538,45.903077,-0.176923,101099.580000,0.236923,5.384615,NaN,NaN,79.461538,6.0
1,2021-01-01-01:00,10.415385,50.623077,0.413077,101117.731538,1.106154,109.230769,NaN,NaN,10.692308,6.0
2,2021-01-01-02:00,9.307692,55.988462,0.879231,101154.028462,0.118462,23.846154,NaN,NaN,61.000000,6.0
3,2021-01-01-03:00,8.753846,58.390000,0.922308,101161.810000,1.265385,103.846154,NaN,NaN,193.538462,6.0
4,2021-01-01-04:00,8.407692,59.661538,0.923846,101203.300769,0.947692,80.000000,NaN,NaN,192.923077,6.0


In [45]:
file_name = os.path.join('epw-raw', 'solar-parameters-la.csv')
df_solar = pd.read_csv(file_name, sep=',', parse_dates=True, infer_datetime_format=True, encoding='UTF-8')
df_solar.describe()
for field in ['A', 'B', 'C', 'AVSC', 'cos-zenith']:
    df_csv_result[field] = df_solar[field]

In [46]:
# write EPW
data_to_write = dict()
for i, row in df_csv_result.iterrows():
#     row['hum_rat'] = psychropy.Hum_rat2(float(row['dry_bulb']), float(row['hurs']), float(row['ps']))
#     row['dew'] = psychropy.Dew_point(float(row['ps']), float(row['hum_rat']))
    mon = int(row['date_time'].split('-')[1])
    day = int(row['date_time'].split('-')[2])
    hour = int(row['date_time'].split('-')[3].split(':')[0])
    row['year'] = row['date_time'].split('-')[0]
    data_to_write[str(mon) + "/" + str(day) + "/" + str(hour)] = row

In [47]:
data_to_write['1/1/0']

date_time                2021-01-01-00:00
dry_bulb                        11.161538
rel_humi                        45.903077
dew_point                       -0.176923
pressure                        101099.58
wind_speed                       0.236923
wind_dir                         5.384615
wind_chill                            NaN
heat_index                            NaN
cloud_layer_1                   79.461538
cloud_layer_1_digit_m                 6.0
A                                 1232.83
B                                0.141412
C                                0.056717
AVSC                              1.03348
cos-zenith                      -0.974462
year                                 2021
Name: 0, dtype: object

In [48]:
import math
f = open(EPW_FILE, 'w')

with open(LOC_WEATHER_FILE, newline='') as csvfile:
    csvreader = csv.reader(csvfile, delimiter=',', quotechar='|')
    for row in csvreader:
        if (len(row) == 35):
            mon = row[1]
            day = row[2]
            hour = row[3]
            minute = row[4]
            # parse 8/1/18 0:00 to 7/31/18 24:00
            if int(hour) == 24:
                hour = 0
                if int(day) == days_in_mon[int(mon)]:
                    mon = int(mon) + 1
                    day = 1
                else:
                    day = int(day) + 1

            # Notes: The format of the CSV time stamp
            cur_time_stamp = str(mon) + "/" + str(day) + "/" + str(hour)
            if cur_time_stamp in data_to_write:
                cur_data_to_write = data_to_write[cur_time_stamp]
                row[0] = '2021'
                row[0] = str(cur_data_to_write['year'])
                # calculate solar items using clout cover data
                if cloud_cover_header in cur_data_to_write:
                    if np.isnan(cur_data_to_write[cloud_cover_header]):
                        cloud_cover_percent = int(row[23]) / 10.0
                    else:
                        cloud_cover_percent = (cur_data_to_write[cloud_cover_header] - 1) / 9.0
                    cloud_cover_tenth = int(cloud_cover_percent * 10)
                    (row[14], row[15]) = acs.ashrae_clear_sky(cloud_cover_percent, 
                                                              cur_data_to_write['cos-zenith'],
                                                              cur_data_to_write['A'],
                                                              cur_data_to_write['B'],
                                                              cur_data_to_write['C'],
                                                              cur_data_to_write['AVSC'])
                    row[12] = '9999'
                    row[23] = str(cloud_cover_tenth)
#                 if 'is_rain' in cur_data_to_write:
#                     if cur_data_to_write['is_rain'] == 1:
#                         row[26] = '0'
#                         row[27] = '919999999'
                for epw_col in csv_headers_epw_map:
                    if csv_headers_epw_map[epw_col] in cur_data_to_write:
                        try:
                            row[epw_col] = str(round(cur_data_to_write[csv_headers_epw_map[epw_col]], 3))
                        except:
                            row[epw_col] = str(cur_data_to_write[csv_headers_epw_map[epw_col]])
            else:
                print(cur_time_stamp + ' not found.')
        f.write(','.join(row) + '\n')

f.close()

13/1/0 not found.
